In [1]:
# only for testing
# use main.py for training


# things to improve performance (speed)
# use torch.utils.random_split https://www.programcreek.com/python/example/125046/torch.utils.data.random_split
# https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html?highlight=checkpoint
# implement "channels Last Memory Format" (beta) https://pytorch.org/tutorials/intermediate/memory_format_tutorial.html

In [2]:
# sample image
import torch

from dataset.utils import pNormalize
from dataset.datasets import sentinel
from model.models import UNET

from torchvision import transforms
from torch.utils.data import DataLoader
q_hi = torch.tensor([2102.0, 1716.0, 1398.0, 4732.0, 2434.42919921875, 3701.759765625, 4519.2177734375, 4857.7734375, 3799.80322265625, 3008.8935546875])
q_lo = torch.tensor([102.0, 159.0, 107.0, 77.0, 106.98081970214844, 79.00384521484375, 86.18966674804688, 70.40167236328125, 50.571197509765625, 36.95356750488281])
        
norm = pNormalize(
                maxPer=q_hi,
                minPer=q_lo
)


# Create experimental dataset and loader, rgb=True for 3 channels (default = False)
test_set = sentinel(
    root_dir='processed-data/dsen_2_256_new_split', img_transform=norm)
test_loader = DataLoader(test_set, batch_size=10, num_workers=0)


# access one image from dataset
img, label = test_set[0]
#print('image shape:', img.shape)


# or  batch from DataLoader
dataiter = iter(test_loader)
images, labels = dataiter.next()





# define model
model = UNET(in_channels=images.shape[1])
model.load_state_dict(torch.load('model_epoch_14.pt',map_location=torch.device('cpu')))
# disable gradients etc. model.train() for training https://medium.com/jun94-devpblog/pytorch-6-model-train-vs-model-eval-no-grad-hyperparameter-tuning-3812c216a3bd
model.eval()

output = model(images)

#pred = torch.argmax(output, dim=1)
     # prediction
pred= torch.nn.functional.softmax(output,dim=1)
pred = torch.argmax(pred,dim=1)


#print('prediction:', pred.shape)
#print('labels:', labels.shape)

print('output:',output.shape)
print('labl', labels.shape)
print('pred:',pred.shape)

output: torch.Size([10, 28, 256, 256])
labl torch.Size([10, 256, 256])
pred: torch.Size([10, 256, 256])


In [1]:
import matplotlib.pyplot as plt
import torch 

def plot_pred_lbl_rgb(pred,labl,rgb):
    
    # def fig
    fig = plt.figure(figsize=(10, 7))
    rows = 1
    columns = 3
    
    # Adds a subplot at the 1st position
    fig.add_subplot(rows, columns, 1)
    plt.imshow(pred)
    
    plt.title("Pred (%d classes)"%(len(torch.unique(pred)))) 
    
    # Adds a subplot at the 2nd position
    fig.add_subplot(rows, columns, 2)
    plt.imshow(labl)          
    plt.title("Label (%d classes)"%(len(torch.unique(labl))))
    
    # Adds a subplot at the 3rd position
    fig.add_subplot(rows, columns, 3)
    plt.imshow(rgb.permute(1,2,0).numpy())
    plt.title("RGB")
    plt.axis('off')
    plt.show()

for idx,(images,labels) in enumerate(test_loader):
#dataiter = iter(test_loader)
#images, labels = dataiter.next()
    output = model(images)
    pred= torch.nn.functional.softmax(output,dim=1)
    pred = torch.argmax(pred,dim=1)
    break
    #for i in range(pred.shape[0]):
    #    plot_pred_lbl_rgb(pred[i,:,:],labels[i,:,:], images[i,0:3,:,:]  )
    #if idx==5:
    #    break

NameError: name 'test_loader' is not defined

In [6]:
import torchvision
from datetime import datetime


# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
# visualise images in tensorboard
# Create a grid from the images and show them
# https://pytorch.org/vision/main/auto_examples/plot_visualization_utils.html#sphx-glr-auto-examples-plot-visualization-utils-py
img_grid = torchvision.utils.make_grid(pred.unsqueeze(dim=1))
#show(img_grid)
#matplotlib_imshow(img_grid, one_channel=False)
#print('  '.join(classes[labels[j]] for j in range(4))
# Default log_dir argument is "runs" - but it's good to be specific
# torch.utils.tensorboard.SummaryWriter is imported above
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/visualise_data')

# Write image data to TensorBoard log dir
writer.add_image('Four Sat Images', img_grid)
writer.flush()

# To view, start TensorBoard on the command line with:
#  tensorboard --logdir=runs
# ...and open a browser tab to http://localhost:6006/

#if on remote
## on remote
##tensorboard --logdir <path> --port 6006
## forward everything on port 6006 on server to 
##ssh -L 16006:127.0.0.1:6006 -i vmexjobb_key.pem exjobbare@vdexjobb.westeurope.cloudapp.azure.com

In [ ]:
# run this to make sure tensorboard is working
#!conda list | grep tensorboard
import torch
#!conda install tensorboard -y
from torch.utils.tensorboard import SummaryWriter
print(torch.__version__)

#make sure this executes
SummaryWriter()

1.12.0.dev20220425


In [4]:
#visualise images in tensorboard

dataiter = iter(training_loader)
images, labels = dataiter.next()

# Create a grid from the images and show them
# https://pytorch.org/vision/main/auto_examples/plot_visualization_utils.html#sphx-glr-auto-examples-plot-visualization-utils-py
img_grid = torchvision.utils.make_grid(images)
#show(img_grid)
#matplotlib_imshow(img_grid, one_channel=False)
#print('  '.join(classes[labels[j]] for j in range(4))
# Default log_dir argument is "runs" - but it's good to be specific
# torch.utils.tensorboard.SummaryWriter is imported above
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/visualise_data')

# Write image data to TensorBoard log dir
writer.add_image('Sat Images', img_grid)
writer.flush()

# To view, start TensorBoard on the command line with:
#  tensorboard --logdir=runs
# ...and open a browser tab to http://localhost:6006/

#if on remote
## on remote
##tensorboard --logdir <path> --port 6006
## forward everything on port 6006 on server to 
##ssh -L 16006:127.0.0.1:6006 -i vmexjobb_key.pem exjobbare@vdexjobb.westeurope.cloudapp.azure.com

NameError: name 'training_loader' is not defined

In [ ]:
#visualizing model in tensorboard


# Again, grab a single mini-batch of images
dataiter = iter(training_loader)
images, labels = dataiter.next()

# add_graph() will trace the sample input through your model,
# and render it as a graph.
writer.add_graph(model, images)
writer.flush()

/media/oskar/ESSD/deepSat/model/models.py:56: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.shape != concat_layer.shape:
